In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import csv

Code to pull in data from Wiki page

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

Code to pull in table from Wiki page

In [3]:
table = soup.find('tbody')

Code to create a list with the table headings

In [4]:
row = ['Postcode','Borough','Neighbourhood']

Code to add the data from the table to the list and converts the html data to text

In [5]:
for trs in table.find_all('tr'):
    for td in trs.find_all('td'):
        td_text = td.text
        row.append(td_text)

Code to format the list into a list with groups of three 

In [6]:
n=3
final = [row[i * n:(i + 1) * n] for i in range((len(row) + n - 1) // n )]  

Code to convert the list to a dataframe

In [7]:
df_final = pd.DataFrame(final)
df_final[2] = df_final[2].apply(lambda x:x.strip('\n'))

Code to remove the "Not assigned" rows in the Borough column

In [8]:
df_final = df_final[df_final[1]!='Not assigned']

Code to replace "Not assigned" in Neighborhood column with value in Borough column

In [9]:
df_final[2] = np.where(df_final[2]=='Not assigned', df_final[1], df_final[2])

Code to move the header names from row 0 and makes them the header  

In [10]:
header = df_final.iloc[0]
df_final = df_final[1:]
df_final = df_final.rename(columns = header)

Code that combines Neighborhood names with the same PostalCode and Borough

In [11]:
df_final = df_final.groupby(['Postcode','Borough']).agg({'Neighbourhood': lambda x:','.join(set(x))})

Code that resets the indexes

In [12]:
df_final = df_final.reset_index()

Prints the header and values of the table and prints the shape of table

In [13]:
print(df_final)
df_final.shape

    Postcode           Borough  \
0        M1B       Scarborough   
1        M1C       Scarborough   
2        M1E       Scarborough   
3        M1G       Scarborough   
4        M1H       Scarborough   
5        M1J       Scarborough   
6        M1K       Scarborough   
7        M1L       Scarborough   
8        M1M       Scarborough   
9        M1N       Scarborough   
10       M1P       Scarborough   
11       M1R       Scarborough   
12       M1S       Scarborough   
13       M1T       Scarborough   
14       M1V       Scarborough   
15       M1W       Scarborough   
16       M1X       Scarborough   
17       M2H        North York   
18       M2J        North York   
19       M2K        North York   
20       M2L        North York   
21       M2M        North York   
22       M2N        North York   
23       M2P        North York   
24       M2R        North York   
25       M3A        North York   
26       M3B        North York   
27       M3C        North York   
28       M3H  

(103, 3)

In [37]:
# The code was removed by Watson Studio for sharing.

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [51]:
df_final2 = pd.merge(df_final,df_data,left_on='Postcode',right_on='Postcode')
df_final2 = df_final2.rename(columns={'Latitude_y':'Latitude'})
df_final2 = df_final2.drop('Latitude_x',axis=1)
df_final2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
